In [1]:
import json 
import numpy as np 
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder

In [35]:
#!pip install scikit-learn

In [94]:
with open('intents.json') as file:
    data = json.load(file)

In [95]:
training_sentences = []
training_labels = []
labels = []
responses = []


for intent in data['intents']:
    for pattern in intent['patterns']:
        training_sentences.append(pattern)
        training_labels.append(intent['tag'])
    responses.append(intent['responses'])
    
    if intent['tag'] not in labels:
        labels.append(intent['tag'])
        
num_classes = len(labels)

In [96]:
lbl_encoder = LabelEncoder()
lbl_encoder.fit(training_labels)
training_labels = lbl_encoder.transform(training_labels)


In [97]:
vocab_size = 1000
embedding_dim = 16
max_len = 20
oov_token = "<OOV>"

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token) # adding out of vocabulary token
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded_sequences = pad_sequences(sequences, truncating='post', maxlen=max_len)

In [98]:
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_len))
model.add(GlobalAveragePooling1D())
model.add(Dense(16, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [99]:
model.summary()

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_11 (Embedding)    (None, 20, 16)            16000     
                                                                 
 global_average_pooling1d_11  (None, 16)               0         
  (GlobalAveragePooling1D)                                       
                                                                 
 dense_33 (Dense)            (None, 16)                272       
                                                                 
 dense_34 (Dense)            (None, 16)                272       
                                                                 
 dense_35 (Dense)            (None, 18)                306       
                                                                 
Total params: 16,850
Trainable params: 16,850
Non-trainable params: 0
_________________________________________________

In [100]:
epochs = 550
history = model.fit(padded_sequences, np.array(training_labels), epochs=epochs)

Epoch 1/550
3/3 [==============================] - 0s 1ms/step - loss: 2.8913 - accuracy: 0.0602
Epoch 2/550
3/3 [==============================] - 0s 880us/step - loss: 2.8899 - accuracy: 0.0482
Epoch 3/550
3/3 [==============================] - 0s 991us/step - loss: 2.8888 - accuracy: 0.1084
Epoch 4/550
3/3 [==============================] - 0s 1ms/step - loss: 2.8883 - accuracy: 0.0964
Epoch 5/550
3/3 [==============================] - 0s 1ms/step - loss: 2.8876 - accuracy: 0.0964
Epoch 6/550
3/3 [==============================] - 0s 868us/step - loss: 2.8870 - accuracy: 0.0964
Epoch 7/550
3/3 [==============================] - 0s 902us/step - loss: 2.8863 - accuracy: 0.0964
Epoch 8/550
3/3 [==============================] - 0s 952us/step - loss: 2.8856 - accuracy: 0.0964
Epoch 9/550
3/3 [==============================] - 0s 907us/step - loss: 2.8847 - accuracy: 0.0964
Epoch 10/550
3/3 [==============================] - 0s 883us/step - loss: 2.8841 - accuracy: 0.0964
Epoch 11/550
3/

In [101]:
# saving model
model.save("chat_model.h5")

import pickle

# saving tokenizer
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
    

# saving label encoder
with open('label_encoder.pickle', 'wb') as ecn_file:
    pickle.dump(lbl_encoder, ecn_file, protocol=pickle.HIGHEST_PROTOCOL)